In [2]:
import os
os.chdir("/home/gridsan/qwang/urban-control/")
%load_ext autoreload
%autoreload 2


In [35]:
import os
import math
from PIL import Image

def create_shifted_tiles(base_path, z, x, y, offset_x, offset_y, output_path):
    """
    Create shifted tiles at the same zoom level (z), with paths like:
    - Input:  {base_path}/z+0+0/x/y.png
    - Output: {output_path}/z+{offset_x}+{offset_y}/x/y.png
    """

    # Skip shifts that are too small (<3% of tile size)
    if offset_x < 0.03 and offset_y < 0.03:
        return

    # Open the current tile and its neighbors
    try:
        current_tile = Image.open(f"{base_path}/{z}+0+0/{x}/{y}.png")
        right_tile = Image.open(f"{base_path}/{z}+0+0/{x+1}/{y}.png")
        bottom_tile = Image.open(f"{base_path}/{z}+0+0/{x}/{y+1}.png")
        bottom_right_tile = Image.open(f"{base_path}/{z}+0+0/{x+1}/{y+1}.png")
    except FileNotFoundError as e:
        # print(f"Skipping {z}+0+0/{x}/{y}: Missing neighbor tile")
        return 0

    tile_size = current_tile.width  # Assuming 256x256 tiles
    px_x, px_y = int(offset_x * tile_size), int(offset_y * tile_size)

    # Create the shifted tile by compositing parts from 4 tiles
    new_tile = Image.new("RGB", (tile_size, tile_size))
    
    # Top-left: Current tile (cropped from offset)
    new_tile.paste(
        current_tile.crop((px_x, px_y, tile_size, tile_size)),
        (0, 0)
    )
    # Top-right: Right tile (left side)
    new_tile.paste(
        right_tile.crop((0, px_y, px_x, tile_size)),
        (tile_size - px_x, 0)
    )
    # Bottom-left: Bottom tile (top side)
    new_tile.paste(
        bottom_tile.crop((px_x, 0, tile_size, px_y)),
        (0, tile_size - px_y)
    )
    # Bottom-right: Bottom-right tile (top-left corner)
    new_tile.paste(
        bottom_right_tile.crop((0, 0, px_x, px_y)),
        (tile_size - px_x, tile_size - px_y)
    )

    # Save to {output_path}/z+{offset_x}+{offset_y}/x/y.png
    os.makedirs(f"{output_path}/{z}+{int(offset_x*10)}+{int(offset_y*10)}/{x}", exist_ok=True)
    new_tile.save(f"{output_path}/{z}+{int(offset_x*10)}+{int(offset_y*10)}/{x}/{y}.png")
    # print(f"Saved: {z}+{int(offset_x*10)}+{int(offset_y*10)}/{x}/{y}.png")
    
    return 1

In [ ]:
# la_boxes
# (11179,11228) (26120,26146) - 1274/1274
# (11201,11233) (26157,26193) - 1041/1152
# (11259,11324) (26143,26180) - 2302/2405
# (11256,11317) (26185, 26255) - 3722/4270

# city = "la"
# x1_list = [11179,11201,11259,11256]
# x2_list = [11228,11233,11324,11317]
# y1_list = [26120,26157,26143,26185]
# y2_list = [26146,26193,26180,26255]

# dallas boxes
# (15126,26436) -> (15155,26454) 522/522
# (15125,26458) -> (15153,26486) 1306/1306
# (15118,26458) -> (15153,26493) 1225/1225
city = "dallas"
# x1_list = [15126,15125]
# x2_list = [15155,15153]
# y1_list = [26436,26458]
# y2_list = [26454,26486]
x1_list = [15118]
x2_list = [15153]
y1_list = [26458]
y2_list = [26493]


# chicago boxes
# (16781, 24343) -> (16834, 24414) 3763
# city = 'chicago'
# x1_list = [16781]
# x2_list = [16834]
# y1_list = [24343]
# y2_list = [24414]

total = 0
success = 0
for x1, x2, y1, y2 in zip(x1_list, x2_list, y1_list, y2_list):
    for x in range(x1,x2):
        for y in range(y1,y2):
            for p in [0.3,0.7]:
                success += create_shifted_tiles(f"./data/satellite_tiles/{city}/","16",x,y,p,1-p,f"./data/satellite_tiles/{city}/")
                total += 1

    print(success, '/', total)


2450 / 2450
